ปฏิบัติการครั้งที่ 13 กระบวนวิชา 229351 Statistical Learning for Data Science

คำชี้แจง

1. ให้เริ่มทำปฏิบัติการจาก colab notebook/ipynb ที่กำหนดให้ จากนั้นบันทึกไว้เป็นไฟล์ *.pdf

ดาวน์โหลดข้อมูลจำลองพนักงาน IBM ใน link ข้างล่างนี้  
http://www.donlapark.cmustat.com/229351/data/Attrition.csv  

In [ ]:
# uploading the csv file to colab

!pip install wget

!python -m wget -o Attrition.csv http://www.donlapark.cmustat.com/229351/data/Attrition.csv
!python -m wget -o ordinal_encoder.py http://www.donlapark.cmustat.com/229351/ordinal_encoder.py

# import module ที่ต้องใช้
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:

data = pd.read_csv('Attrition.csv')

print(data.info())

data.head()

###ข้อมูลนี้มีตัวแปรเชิงคุณภาพหลายชนิด มีคำแนะนำในการแปลงข้อมูลก่อนนำไป fit โมเดลดังนี้

* ตัวแปรเชิงปริมาณ ให้ normalize ข้อมูลก่อน (ลบข้อมูลแต่ละตัวแปรด้วย mean ของตัวแปรนั้น แล้วหารด้วย std ของตัวแปรนั้น)
* ตัวแปรเชิงคุณภาพแบบ ordinal (มีลำดับ) ให้แปลงค่าตัวแปรจากน้อยไปมากเป็น 1,2,3,...
* ตัวแปรเชิงคุณภาพแบบ nominal (ไม่มีลำดับ) ให้แปลงแต่ละค่าเป็นเวกเตอร์ที่ประกอบไปด้วย 0 และ 1 อย่างเช่น 
** `Life Sciences -> [0 , 0 , 0]`
** `Marketing -> [1 , 0 , 0]`
** `Medical -> [0 , 1 , 0]`
** `Other -> [0 , 0 , 1]`

ข้างล่างนี้คือตัวอย่างการแปลงข้อมูลด้วยวิธีใน `scikit-learn`

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit_transform(data[["DistanceFromHome", "Age"]])
#scaler.transform(data[["DistanceFromHome", "Age"]])

In [ ]:
from ordinal_encoder import OrdinalEncoder ##use the function ordinal_encoder.py until scikit-learn update their code

print(data["BusinessTravel"])

ord = OrdinalEncoder(categories=[["Non-Travel","Travel_Rarely","Travel_Frequently"]])
ord.fit(data[["BusinessTravel"]])
ord.transform(data[["BusinessTravel"]])


In [ ]:
from sklearn.preprocessing import OneHotEncoder

print(data["EducationField"])

ohe = OneHotEncoder(drop = "first")
ohe.fit(data[["EducationField"]])
ohe.transform(data[["EducationField"]]).toarray()

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from ordinal_encoder import OrdinalEncoder ##use the function ordinal_encoder.py until scikit-learn update their code
from sklearn.compose import ColumnTransformer

num_cols = ["Age", "DailyRate",	"DistanceFromHome", "Education", 
            "EmployeeCount",	"EmployeeNumber", "EnvironmentSatisfaction", 
            "HourlyRate", "JobInvolvement", "JobLevel", 
            "JobSatisfaction",	"MonthlyIncome", "MonthlyRate", 
            "NumCompaniesWorked", "PercentSalaryHike", 
            "PerformanceRating",	"RelationshipSatisfaction", "StandardHours", 
            "StockOptionLevel", "TotalWorkingYears", "TrainingTimesLastYear", 
            "WorkLifeBalance", "YearsAtCompany", "YearsInCurrentRole", 
            "YearsSinceLastPromotion", "YearsWithCurrManager"]
ord_cols = ["BusinessTravel"]
ohe_cols = ["Attrition", "Department", "EducationField", "Gender", "JobRole",
            "MaritalStatus",	"Over18", "OverTime"]

transformer = ColumnTransformer(
    [
     ('num_scaler', StandardScaler(), num_cols),
     ('ord_encoder', OrdinalEncoder(categories=[["Non-Travel","Travel_Rarely","Travel_Frequently"]]), ord_cols),
     ('ohe_encoder', OneHotEncoder(drop='first'), ohe_cols)
     ]
    )

# Can combine two lines below into 'data_prep = transformer.fit_transform(data)'
transformer.fit(data)
data_prep = transformer.transform(data) #change 'data' to your test set

df_prep = pd.DataFrame(data_prep, columns=transformer.get_feature_names_out())

df_prep

In [ ]:
y = df_prep['ohe_encoder__Attrition_Yes']
X = df_prep.drop(['ohe_encoder__Attrition_Yes'], axis = 1)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA()
lda.fit(X,y)
y_pred = lda.predict(X)

In [ ]:
print(lda.means_)
print(lda.priors_)

In [ ]:
from sklearn.metrics import classification_report


print(classification_report(y, y_pred))

#### Exercise 1:
* แบ่งข้อมูลออกเป็น Training set กับ Test set โดยสุ่ม โดยมีข้อมูลอยู่ใน Test set เป็น 30% ของข้อมูลทั้งหมด (ใช้ `from sklearn.model_selection import train_test_split`)
* ทำการแปลงข้อมูล (preprocessing) ตามตัวอย่างข้างบน
* Train โมเดล Logistic Regression และ Linear Distriminant Analysis โดยใช้ Training set
* คำนวณค่า accuracy score และ F1 ของค่า `Attribute = 1` แล้วลองเปรียบเทียบ โมเดลไหนมีการทำนายค่า `Attribute` ที่ดีกว่ากัน?

In [ ]:
# 1. Define transformer
transformer = ColumnTransformer(......)

# 2. Split data with train_test_split
train_data, test_data = ...

# 3. Fit+transform on training set
X_train_prep = transformer.fit_transform(train_data)

# 4. Split transformed data into X and y
y_train = train_data[["Attrition"]]
X_train = ...

# 5. Train model on X_train_prep

# 6. Use the above transformer to transform on test set
X_test_prep = transformer.transform(test_data)

# 7. Split transformed data into X and y
y_test = test_data[["Attrition"]]
X_test = ...

# a given column is not a column of the dataframe